In [1]:

%pprint
import sys
if (osp.join('..', 'py') not in sys.path): sys.path.insert(1, osp.join('..', 'py'))
from jobpostlib import (cu, datetime, duration, freq, hau, humanize, nu, scrfcu, slrcu, su, t0, time, wsu, speech_engine)
import os

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 15 seconds



----

In [12]:

wsu.save_email_to_file('builtin', verbose=True)

['powershell.exe', '-ExecutionPolicy', 'Bypass', '-Command', 'Start-Process "C:\\Program Files\\Notepad++\\notepad++.exe" -ArgumentList "C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\job-hunting\\data\\html\\linkedin_email.html"']


In [8]:

file_path = '../data/html/builtin_email.html'
page_soup = get_page_soup(file_path)
css_selector = 'a'
link_soups_list = page_soup.select(css_selector)
for link_obj in link_soups_list:
    print(link_obj['href'])

https://u18536776.ct.sendgrid.net/ss/c/mTtJyuGHfDoEKQLfQ5xsFJSq7a_pmlcB60T6Q8DuHdkrGqM8uAwmZNUbo4MeNQP8uezdHn8RhPbDThAMj5pnbXhIWVM4g8iclH6r2ZQrjw0rc5SWackO8KETlYXPUTUl_ZSnfD6WTtg95ZnbO0uKFE7NBowDH89GFQt8ZsD_co_Ojc22PydHqTdTlmZ4-EWC19E1J428s7_JlBNC6a318HpYLOaeR_nN9PDcL-qhaAs/3ng/AtxM_xqtRdeo7jNTWLmkZw/h35/Lm1tanbxXoAP5uE19hBqMD8SPbdDXA13Kr9ySLHGcrI
https://u18536776.ct.sendgrid.net/ss/c/mTtJyuGHfDoEKQLfQ5xsFBJ564RaRzx1yWhvIKBx6yTBBpPxKvF0aKxKLFZyE_v21mOnyt_XFcl_wasijRlH4tLXEF6DFp9oEcg_LOciTNf6P0qjZZS_nzT0z6yBOOy7uE4sbC3gVm65J80zACejVuTP7PRbp7KnisnqCYD6zBS6geqTO0eFkbG85mWz1XClwG9UsKT8t1ZoDX-knJiIHhNncY4qiWu4pnuYqZH5Pss/3ng/AtxM_xqtRdeo7jNTWLmkZw/h36/k6QRPK-TFdXnXOK7-JAU5ZedTzzIH0x09WhE0blZNF4
https://u18536776.ct.sendgrid.net/ss/c/mTtJyuGHfDoEKQLfQ5xsFJSq7a_pmlcB60T6Q8DuHdkgF7sXzTKz1a95ECD4GjgSVARynltc2M-7_u8dYZfcRy3rvFfTWAET0HGynUjtDSdJZ8aS59bWkkPHOkDhBhuPHXKc8KtaZcQmTxbWLhEPjHRlDbqh7ueWMMB_lfJ5xIBj2GOX9Xx4P9Qf1vPklj6o6uyexB8qhVY5LSPzG8KCcQ/3ng/AtxM_xqtRdeo7jNTWLmkZw/h37/e9xZV-WaHccWLBQ

In [9]:

files_list = []
for link_soup in link_soups_list:
    url_str = link_soup['href']
    try:
        page_soup = get_page_soup(url_str)
        page_title = page_soup.find('title').string.strip()
        file_name = re.sub(r'[^A-Za-z0-9]+', ' ', page_title).strip().replace(' ', '_')
        jk = parse_qs(urlparse(url_str).query).get('jk', [''])[0]
        if len(jk):
            file_name = f'{jk}_{file_name}.html'
        else:
            # file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}.html'
            file_name = f'{file_name}.html'
        file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
        if not os.path.isfile(file_path):
            with open(file_path, 'w', encoding=s.encoding_type) as f:
                print(f'Saving to {file_path}')
                f.write('<html><head><title>')
                f.write(page_title)
                f.write('</title></head><body>')
                row_div_list = page_soup.find_all(name='div', attrs={'class': ['jobsearch-JobComponent-description']})
                for div_soup in row_div_list:
                    f.write(str(div_soup))
                f.write('</body></html>')
            files_list.append(file_name)
        cu.ensure_filename(file_name, verbose=False)
        
        def do_cypher_tx(tx, file_name, url_str, verbose=False):
            cypher_str = """
                MATCH (fn:FileNames {file_name: $file_name})
                SET fn.posting_url = $url_str
                RETURN fn;"""
            if verbose:
                clear_output(wait=True)
                print(cypher_str.replace('$file_name', f'"{file_name}"').replace('$url_str', f'"{url_str}"'))
            tx.run(cypher_str, file_name=file_name, url_str=url_str)
        
        with cu.driver.session() as session:
            session.write_transaction(do_cypher_tx, file_name=file_name, url_str=url_str, verbose=False)
    except HTTPError as e:
        print(f'Got an HTTPError with {url_str}: {str(e).strip()}')
    except URLError as e:
        print(f'Got an URLError with {url_str}: {str(e).strip()}')
    except Exception as e:
        print(f'Got an {e.__class__.__name__} error with {url_str}: {str(e).strip()}')
print('Filing complete')

Filing complete


In [10]:

cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)

In [11]:

files_list

[]